<h1>Trabajdo Realizado para la Feria</h1>

<h2>Integrantes</h2>
<p>Luis Caballero 8-9231440<br>
Ángel Cilli E-8-182257<br>
Jaime Acosta 8-971-833</p>

<p><b>Idea o Concepto:</b>Trabajaremos un reconocedor de "personal" dentro del equipo, un Aruco distinto para Jaime Acosta y Luis Caballero, que cuando sean mostrados a la cámara el programa ejecutará distintos enfoques con Aruco.</p>

<h2>Detector del Personal</h2>

<p>Para el detector de personal usaremos el diccionario <b>DISC_4x4_1000</b>b><br>
Marcado ID 15 → Luis Caballero<br>
Marcador ID 13 → Jaime Acosta</p>

<h1>Programa Limpio/Completo</h1>

In [18]:
#Importaciones 
import cv2
import cv2.aruco as aruco
import numpy as np

# Funcion de mensjae de bienvenida
def bienvenida(name, frame):
    cv2.putText(img=frame, text=f"Bienvenido, {name} Acceso permitido.",
                org=(10, 30), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=0.6,
                color=(255, 255, 255), thickness=2)

# Funcion para control del tiempo en pantalla
def delay(window_name, frame, delay):
    cv2.imshow(window_name, frame)
    cv2.waitKey(delay * 1000)
    cv2.destroyAllWindows()

# Funcion para el video en el marcador
def overlay_video(pts_1, overlay_image, image):
    pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                        [0, overlay_image.shape[0]]])
    M = cv2.getPerspectiveTransform(pts_2, pts_1)
    dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))
    dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)
    image_masked = cv2.bitwise_and(image, image, mask=mask)
    result = cv2.add(dst_image, image_masked)
    return result

# Funcion para la mascara en la cara
def meme(frame):
    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    img_cuerno = cv2.imread('Img/meme.png', 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        x1 = x
        x2 = x + w
        y1 = y - h
        y2 = y + h
        cuerno_resized = cv2.resize(img_cuerno, (w, h))
        frame[y:y + h, x:x + w] = cv2.addWeighted(frame[y:y + h, x:x + w], 1, cuerno_resized, 1, 0)
    return frame

# Main
def main():
    cap = cv2.VideoCapture(0)
    principal_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)# dicionario para el acceso
    luis_dict_5x5 = aruco.getPredefinedDictionary(aruco.DICT_5X5_50)#diccionario para video en marcador
    jaime_dict_6x6 = aruco.getPredefinedDictionary(aruco.DICT_6X6_50)#diccioanrio para mascaras en a cara
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    video_path = 'Img/muchacho.mp4'
    cap_overlay = cv2.VideoCapture(video_path)
    overlay_image_path = 'Img/sigma.png'
    overlay_image = cv2.imread(overlay_image_path, cv2.IMREAD_UNCHANGED)
    overlay_image = cv2.resize(overlay_image, (100, 100))

    def get_next_overlay_frame():
        ret, frame = cap_overlay.read()
        if not ret:
            cap_overlay.set(cv2.CAP_PROP_POS_FRAMES, 0)
            ret, frame = cap_overlay.read()
        return frame

    def draw_augmented_overlay(pts_1, overlay_image, image):
        pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                            [0, overlay_image.shape[0]]])
        cv2.rectangle(overlay_image, (0, 0), (overlay_image.shape[1], overlay_image.shape[0]), (255, 255, 0), 10)
        M = cv2.getPerspectiveTransform(pts_2, pts_1)
        dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))
        dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)
        image_masked = cv2.bitwise_and(image, image, mask=mask)
        result = cv2.add(dst_image, image_masked)
        return result

    while True:
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 15 in ids:
            bienvenida("Luis", frame)
            delay("Diccionario 5x5 - Video en Marcaror", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                corners, ids, _ = aruco.detectMarkers(gray, luis_dict_5x5)

                if ids is not None and 10 in ids:
                    overlay_frame = get_next_overlay_frame()
                    if overlay_frame is not None:
                        frame = draw_augmented_overlay(corners[0][0], overlay_frame, frame)

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break

                cv2.imshow("Diccionario 5x5 - Video en Marcaror", frame)

                if cv2.waitKey(1) & 0xFF == 27:
                    break

        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 13 in ids:
            bienvenida("Jaime", frame)
            delay("Diccionario 6x6 - Mascara en la Cara", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                corners, ids, _ = aruco.detectMarkers(gray, jaime_dict_6x6)

                if ids is not None and 20 in ids:
                    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                    for (x, y, w, h) in faces:
                        face = frame[y:y + h, x:x + w]
                        face = cv2.GaussianBlur(face, (99, 99), 30)
                        frame[y:y + h, x:x + w] = face
                        face_height, face_width, _ = face.shape
                        overlay_image_resized = cv2.resize(overlay_image, (face_width, face_height))
                        frame[y:y + h, x:x + w] = cv2.addWeighted(frame[y:y + h, x:x + w], 1.0, overlay_image_resized[:, :, :3], 0.7, 0)

                elif ids is not None and 8 in ids:
                    meme(frame)

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break

                cv2.imshow("Diccionario 6x6 - Mascara en la Cara", frame)

                if cv2.waitKey(1) & 0xFF == 27:
                    break

        cv2.imshow("Original", frame)

        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cap_overlay.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


<h1>Historial de códigos</h1>

<p>En esta parte encontrará los programas que se hicieron para in probando cada una de las partes que se desea hacer, antes de realizar o unirlo en un unico codigo limpio</p>

In [3]:
#IMPORTACIONES
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import pickle

#AruCO

OVERLAY_SIZE_PER = 1

# Check for camera calibration data
if not os.path.exists('calibration.pckl'):
    print("Necesitas Calibrar tu Cámara para un buen uso")
    exit()
else:
    f = open('calibration.pckl', 'rb')
    cameraMatrix, distCoeffs = pickle.load(f)
    f.close()
    if cameraMatrix is None or distCoeffs is None:
        print("Alguien salio mal. Recalibra tu cámara")
        exit()


# Create the ArUco dictionary and parameters:
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_1000) #Detector de Personal
aruco_dict2 = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_1000) #Imagen 3D
aruco_dict3 = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_1000) #Filtros
parameters = cv2.aruco.DetectorParameters()

# Create the video capture object:
video_capture = cv2.VideoCapture(0)
frame_index = 0;

while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray_frame, aruco_dict, parameters=parameters)

    if ids is not None:
        for i in range(len(ids)):
            if ids[i] == 15:
                # Mensaje de bienvenida para Luis Caballero
                cv2.putText(img=frame, text="Bienvenido Luis", org=(10, 30),
            fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1,
            color=(102, 102, 255), thickness=2)
                #aqui deberia ir lo de filtros o video
            
            elif ids[i] == 13:
                # Mensaje de bienvenida para Jaime Acosta
                cv2.putText(img=frame, text="Bienvenido Jaime", org=(10, 30),
            fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1,
            color=(102, 102, 255), thickness=2)
                #aqui deberia ir lo del 3D
                
            else:
                #Mensaje de no esta registrado
                cv2.putText(img=frame, text="Quien tu eres!?", org=(10, 30),
            fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1,
            color=(102, 102, 255), thickness=2)
                

    cv2.aruco.drawDetectedMarkers(frame, corners, ids)
    cv2.imshow('ArUco Marker and Face Detection', frame)
    

    if cv2.waitKey(20) & 0xFF == ord('c'):
        frame_name = "out/Parcial_frame_{}.png".format(frame_index)
        cv2.imwrite(frame_name, frame)
        frame_index += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


<hr>

In [18]:
# Import required packages
import cv2
import os
import pickle
import numpy as np

OVERLAY_SIZE_PER = 1

# Check for camera calibration data
if not os.path.exists('calibration.pckl'):
    print("You need to calibrate the camera before")
    exit()
else:
    f = open('calibration.pckl', 'rb')
    cameraMatrix, distCoeffs = pickle.load(f)
    f.close()
    if cameraMatrix is None or distCoeffs is None:
        print("Something went wrong. Recalibrate the camera")
        exit()

# Load the video overlay:
video_path = "Img/SheidexPapure.mp4"  # Cambia a la ruta de tu video
cap_overlay = cv2.VideoCapture(video_path)

# Function to get the next frame from the overlay video
def get_next_overlay_frame():
    ret, frame = cap_overlay.read()
    if not ret:
        cap_overlay.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, frame = cap_overlay.read()
    return frame

def draw_augmented_overlay(pts_1, overlay_image, image):
    """Overlay the image 'overlay_image' onto the image 'image'"""

    # Define the squares of the overlay_image image to be drawn:
    pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                        [0, overlay_image.shape[0]]])

    # Draw border to see the limits of the image:
    cv2.rectangle(overlay_image, (0, 0), (overlay_image.shape[1], overlay_image.shape[0]), (255, 255, 0), 10)

    # Create the transformation matrix:
    M = cv2.getPerspectiveTransform(pts_2, pts_1)

    # Transform the overlay_image image using the transformation matrix M:
    dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))

    # Create the mask:
    dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)

    # Compute bitwise conjunction using the calculated mask:
    image_masked = cv2.bitwise_and(image, image, mask=mask)

    # Add the two images to create the resulting image:
    result = cv2.add(dst_image, image_masked)
    return result


# Create the dictionary object and the parameters:
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_7X7_250)
parameters = cv2.aruco.DetectorParameters()
detector = cv2.aruco.ArucoDetector(dictionary, parameters)

# Create video capture object 'capture' to be used to capture frames from the first connected camera:
capture = cv2.VideoCapture(0)

while True:
    # Capture frame by frame from the video capture object 'capture':
    ret, frame = capture.read()

    # We convert the frame to grayscale:
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect markers:
    corners, ids, rejectedImgPoints = detector.detectMarkers(gray_frame)

    # Draw detected markers:
    frame = cv2.aruco.drawDetectedMarkers(image=frame, corners=corners, ids=ids, borderColor=(0, 255, 0))

    if ids is not None:
        # rvecs and tvecs are the rotation and translation vectors respectively, for each of the markers in corners.
        rvecs, tvecs, _ = cv2.aruco.estimatePoseSingleMarkers(corners, 1, cameraMatrix, distCoeffs)

        for rvec, tvec in zip(rvecs, tvecs):
            # Note: The marker coordinate system is centered on the center of the marker
            # The coordinates of the four corners of the marker (in its own coordinate system) are:
            # 1: (-markerLength/2, markerLength/2, 0)
            # 2: (markerLength/2, markerLength/2, 0)
            # 3: (markerLength/2, -markerLength/2, 0)
            # 4: (-markerLength/2, -markerLength/2, 0)
            # Define the points where you want the image to be overlaid (remember: marker coordinate system):
            desired_points = np.float32(
                [[-1 / 2, 1 / 2, 0], [1 / 2, 1 / 2, 0], [1 / 2, -1 / 2, 0], [-1 / 2, -1 / 2, 0]]) * OVERLAY_SIZE_PER

            # Project the points:
            projected_desired_points, jac = cv2.projectPoints(desired_points, rvecs, tvecs, cameraMatrix, distCoeffs)

            # Get the next frame from the overlay video
            overlay_frame = get_next_overlay_frame()

            # Overlay the video frame onto the camera frame
            frame = draw_augmented_overlay(projected_desired_points, overlay_frame, frame)

    # Display the resulting augmented frame:
    cv2.imshow('frame', frame)

    # Press 'q' to exit:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything:
capture.release()
cap_overlay.release()
cv2.destroyAllWindows()


In [6]:
import cv2
import cv2.aruco as aruco
import numpy as np

def welcome_message(name, frame):
    cv2.putText(img=frame, text=f"Bienvenido, {name} Acceso permitido.",
                org=(10, 30), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=0.6,
                color=(255, 255, 255), thickness=2)

def main():
    # Configuración de la cámara
    cap = cv2.VideoCapture(0)

    # Diccionario 4x4 para Luis
    principal_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
    
    # Diccionario 5x5 para Luis
    luis_dict_5x5 = aruco.getPredefinedDictionary(aruco.DICT_5X5_50)

    # Diccionario 6x6 para Jaime
    jaime_dict_6x6 = aruco.getPredefinedDictionary(aruco.DICT_6X6_50)

    # Cargamos el clasificador de caras de OpenCV
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while True:
        ret, frame = cap.read()

        # Convertir la imagen a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar marcadores 4x4
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 15 in ids:
            welcome_message("Luis", frame)
            
            # Abrir nueva ventana para el diccionario 5x5
            cv2.destroyAllWindows()
            cv2.namedWindow("Diccionario 5x5") #Cambiar 

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 5x5
                corners, ids, _ = aruco.detectMarkers(gray, luis_dict_5x5)

                if ids is not None and 10 in ids:
                    # Aplicar blur en la cara
                    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)) #en vez de ser borrado, podria ser marcaras
                    for (x, y, w, h) in faces:
                        face = frame[y:y + h, x:x + w]
                        face = cv2.GaussianBlur(face, (99, 99), 30)
                        frame[y:y + h, x:x + w] = face

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 5x5", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        # Detectar marcadores 6x6 para Jaime
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 13 in ids:
            welcome_message("Jaime", frame)

            # Abrir nueva ventana para el marcador 20
            cv2.destroyAllWindows()
            cv2.namedWindow("Diccionario 6x6")

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 6x6
                corners, ids, _ = aruco.detectMarkers(gray, jaime_dict_6x6)

                if ids is not None and 20 in ids: #Qui se puede colocar otro
                    # Colocar la imagen en la cara
                    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                    for (x, y, w, h) in faces:
                        # Coordenadas de la región de la cara
                        face_region = frame[y:y + h, x:x + w]

                        # Cargar la imagen
                        image = cv2.imread("Img/yemil.jpg")

                        # Verificar si la imagen se cargó correctamente
                        if image is not None:
                            # Ajustar el tamaño de la imagen al tamaño de la cara
                            image = cv2.resize(image, (w, h))

                            # Superponer la imagen en la región de la cara
                            frame[y:y + h, x:x + w] = cv2.addWeighted(face_region, 1, image, 0.7, 0)

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 6x6", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        cv2.imshow("Original", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


V2

In [12]:
import cv2
import cv2.aruco as aruco
import numpy as np
import time

def bienvenida(name, frame):
    cv2.putText(img=frame, text=f"Bienvenido, {name} Acceso permitido.",
                org=(10, 30), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=0.6,
                color=(255, 255, 255), thickness=2)

def delay(window_name, frame, delay):
    cv2.imshow(window_name, frame)
    cv2.waitKey(delay * 1000)
    cv2.destroyAllWindows()

def main():
    # Configuración de la cámara
    cap = cv2.VideoCapture(0)

    # Diccionario 4x4 para Luis
    principal_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
    
    # Diccionario 5x5 para Luis
    luis_dict_5x5 = aruco.getPredefinedDictionary(aruco.DICT_5X5_50)

    # Diccionario 6x6 para Jaime
    jaime_dict_6x6 = aruco.getPredefinedDictionary(aruco.DICT_6X6_50)

    # Cargamos el clasificador de caras de OpenCV
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while True:
        ret, frame = cap.read()

        # Convertir la imagen a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar marcadores 4x4
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 15 in ids:
            bienvenida("Luis", frame)
            delay("Diccionario 5x5", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 5x5
                corners, ids, _ = aruco.detectMarkers(gray, luis_dict_5x5)

                if ids is not None and 10 in ids:
                    # Aplicar blur en la cara
                    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                    for (x, y, w, h) in faces:
                        face = frame[y:y + h, x:x + w]
                        face = cv2.GaussianBlur(face, (99, 99), 30)
                        frame[y:y + h, x:x + w] = face

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 5x5", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        # Detectar marcadores 6x6 para Jaime
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 13 in ids:
            bienvenida("Jaime", frame)
            delay("Diccionario 6x6", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 6x6
                corners, ids, _ = aruco.detectMarkers(gray, jaime_dict_6x6)

                if ids is not None and 20 in ids:
                    # Colocar la imagen en la cara
                    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                    for (x, y, w, h) in faces:
                        # Coordenadas de la región de la cara
                        face_region = frame[y:y + h, x:x + w]

                        # Cargar la imagen
                        image = cv2.imread("Img/yemil.jpg")

                        # Verificar si la imagen se cargó correctamente
                        if image is not None:
                            # Ajustar el tamaño de la imagen al tamaño de la cara
                            image = cv2.resize(image, (w, h))

                            # Superponer la imagen en la región de la cara
                            frame[y:y + h, x:x + w] = cv2.addWeighted(face_region, 1, image, 0.7, 0)

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 6x6", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        cv2.imshow("Original", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


intento de que cuando entra como jaime muestre un video

In [23]:
import cv2
import cv2.aruco as aruco
import numpy as np
import time

def bienvenida(name, frame):
    cv2.putText(img=frame, text=f"Bienvenido, {name} Acceso permitido.",
                org=(10, 30), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=0.6,
                color=(255, 255, 255), thickness=2)

def delay(window_name, frame, delay):
    cv2.imshow(window_name, frame)
    cv2.waitKey(delay * 1000)
    cv2.destroyAllWindows()

def overlay_video(pts_1, overlay_image, image):
    """Overlay the video 'overlay_image' onto the image 'image'"""

    # Define the squares of the overlay_image image to be drawn:
    pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                        [0, overlay_image.shape[0]]])

    # Create the transformation matrix:
    M = cv2.getPerspectiveTransform(pts_2, pts_1)

    # Transform the overlay_image image using the transformation matrix M:
    dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))

    # Create the mask:
    dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)

    # Compute bitwise conjunction using the calculated mask:
    image_masked = cv2.bitwise_and(image, image, mask=mask)

    # Add the two images to create the resulting image:
    result = cv2.add(dst_image, image_masked)
    return result

def main():
    # Configuración de la cámara
    cap = cv2.VideoCapture(0)

    # Diccionario 4x4 para Luis
    principal_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

    # Diccionario 5x5 para Luis
    luis_dict_5x5 = aruco.getPredefinedDictionary(aruco.DICT_5X5_50)

    # Diccionario 6x6 para Jaime
    jaime_dict_6x6 = aruco.getPredefinedDictionary(aruco.DICT_6X6_50)

    # Cargamos el clasificador de caras de OpenCV
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Cargar el video
    video_path = 'Img/SheidexPapure.mp4'  # Cambia esto por la ruta de tu video
    cap_overlay = cv2.VideoCapture(video_path)


    while True:
        ret, frame = cap.read()

        # Convertir la imagen a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar marcadores 4x4
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 15 in ids:
            bienvenida("Luis", frame)
            delay("Diccionario 5x5", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 5x5
                corners, ids, _ = aruco.detectMarkers(gray, luis_dict_5x5)

                if ids is not None and 10 in ids:
                    # Get the next frame from the overlay video
                    ret, overlay_frame = cap_overlay.read()
                    if ret:
                        # Overlay the video frame onto the camera frame
                        frame = overlay_video(corners[0][0], overlay_frame, frame)

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 5x5", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        # Detectar marcadores 6x6 para Jaime
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 13 in ids:
            bienvenida("Jaime", frame)
            delay("Diccionario 6x6", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 6x6
                corners, ids, _ = aruco.detectMarkers(gray, jaime_dict_6x6)

                if ids is not None and 20 in ids:
                    # Colocar la imagen en la cara
                    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                    for (x, y, w, h) in faces:
                        # Coordenadas de la región de la cara
                        face_region = frame[y:y + h, x:x + w]

                        # Cargar la imagen
                        image = cv2.imread("Img/yemil.jpg")

                        # Verificar si la imagen se cargó correctamente
                        if image is not None:
                            # Ajustar el tamaño de la imagen al tamaño de la cara
                            image = cv2.resize(image, (w, h))

                            # Superponer la imagen en la región de la cara
                            frame[y:y + h, x:x + w] = cv2.addWeighted(face_region, 1, image, 0.7, 0)

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 6x6", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        cv2.imshow("Original", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
            break

    cap.release()
    cap_overlay.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


<h1>V#4</h1>

<p>Esta parte ya logra el acceso por aruco de Luis y Jaime, y con el de Luis ya logra reproducir un video</p>

In [25]:
import cv2
import cv2.aruco as aruco
import numpy as np
import time

def bienvenida(name, frame):
    cv2.putText(img=frame, text=f"Bienvenido, {name} Acceso permitido.",
                org=(10, 30), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=0.6,
                color=(255, 255, 255), thickness=2)

def delay(window_name, frame, delay):
    cv2.imshow(window_name, frame)
    cv2.waitKey(delay * 1000)
    cv2.destroyAllWindows()

def overlay_video(pts_1, overlay_image, image):
    """Overlay the video 'overlay_image' onto the image 'image'"""

    # Define the squares of the overlay_image image to be drawn:
    pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                        [0, overlay_image.shape[0]]])

    # Create the transformation matrix:
    M = cv2.getPerspectiveTransform(pts_2, pts_1)

    # Transform the overlay_image image using the transformation matrix M:
    dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))

    # Create the mask:
    dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)

    # Compute bitwise conjunction using the calculated mask:
    image_masked = cv2.bitwise_and(image, image, mask=mask)

    # Add the two images to create the resulting image:
    result = cv2.add(dst_image, image_masked)
    return result

def main():
    # Configuración de la cámara
    cap = cv2.VideoCapture(0)

    # Diccionario 4x4 para Luis
    principal_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

    # Diccionario 5x5 para Luis
    luis_dict_5x5 = aruco.getPredefinedDictionary(aruco.DICT_5X5_50)

    # Diccionario 6x6 para Jaime
    jaime_dict_6x6 = aruco.getPredefinedDictionary(aruco.DICT_6X6_50)

    # Cargamos el clasificador de caras de OpenCV
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Cargar el video
    video_path = 'Img/SheidexPapure.mp4'  # Cambia esto por la ruta de tu video
    cap_overlay = cv2.VideoCapture(video_path)

    def get_next_overlay_frame():
        ret, frame = cap_overlay.read()
        if not ret:
            cap_overlay.set(cv2.CAP_PROP_POS_FRAMES, 0)
            ret, frame = cap_overlay.read()
        return frame

    def draw_augmented_overlay(pts_1, overlay_image, image):
        """Overlay the image 'overlay_image' onto the image 'image'"""

        # Define the squares of the overlay_image image to be drawn:
        pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                            [0, overlay_image.shape[0]]])

        # Draw border to see the limits of the image:
        cv2.rectangle(overlay_image, (0, 0), (overlay_image.shape[1], overlay_image.shape[0]), (255, 255, 0), 10)

        # Create the transformation matrix:
        M = cv2.getPerspectiveTransform(pts_2, pts_1)

        # Transform the overlay_image image using the transformation matrix M:
        dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))

        # Create the mask:
        dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)

        # Compute bitwise conjunction using the calculated mask:
        image_masked = cv2.bitwise_and(image, image, mask=mask)

        # Add the two images to create the resulting image:
        result = cv2.add(dst_image, image_masked)
        return result

    while True:
        ret, frame = cap.read()

        # Convertir la imagen a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar marcadores 4x4
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 15 in ids:
            bienvenida("Luis", frame)
            delay("Diccionario 5x5", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 5x5
                corners, ids, _ = aruco.detectMarkers(gray, luis_dict_5x5)

                if ids is not None and 10 in ids:
                    # Get the next frame from the overlay video
                    overlay_frame = get_next_overlay_frame()
                    if overlay_frame is not None:
                        # Overlay the video frame onto the camera frame
                        frame = draw_augmented_overlay(corners[0][0], overlay_frame, frame)

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 5x5", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        # Detectar marcadores 6x6 para Jaime
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 13 in ids:
            bienvenida("Jaime", frame)
            delay("Diccionario 6x6", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 6x6
                corners, ids, _ = aruco.detectMarkers(gray, jaime_dict_6x6)

                if ids is not None and 20 in ids:
                    # Colocar la imagen en la cara
                    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                    for (x, y, w, h) in faces:
                        # Coordenadas de la región de la cara
                        face_region = frame[y:y + h, x:x + w]

                        # Cargar la imagen
                        image = cv2.imread("Img/yemil.jpg")

                        # Verificar si la imagen se cargó correctamente
                        if image is not None:
                            # Ajustar el tamaño de la imagen al tamaño de la cara
                            image = cv2.resize(image, (w, h))

                            # Superponer la imagen en la región de la cara
                            frame[y:y + h, x:x + w] = cv2.addWeighted(face_region, 1, image, 0.7, 0)

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 6x6", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        cv2.imshow("Original", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
            break

    cap.release()
    cap_overlay.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


<h1>V#5</h1>

<p>cuando entra como Jaime, debe ejecutar un filtro de payaso</p>

In [12]:
import cv2
import cv2.aruco as aruco
import numpy as np
import time

def bienvenida(name, frame):
    cv2.putText(img=frame, text=f"Bienvenido, {name} Acceso permitido.",
                org=(10, 30), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=0.6,
                color=(255, 255, 255), thickness=2)

def delay(window_name, frame, delay):
    cv2.imshow(window_name, frame)
    cv2.waitKey(delay * 1000)
    cv2.destroyAllWindows()

def overlay_video(pts_1, overlay_image, image):
    """Overlay the video 'overlay_image' onto the image 'image'"""

    # Define the squares of the overlay_image image to be drawn:
    pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                        [0, overlay_image.shape[0]]])

    # Create the transformation matrix:
    M = cv2.getPerspectiveTransform(pts_2, pts_1)

    # Transform the overlay_image image using the transformation matrix M:
    dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))

    # Create the mask:
    dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)

    # Compute bitwise conjunction using the calculated mask:
    image_masked = cv2.bitwise_and(image, image, mask=mask)

    # Add the two images to create the resulting image:
    result = cv2.add(dst_image, image_masked)
    return result

def main():
    # Configuración de la cámara
    cap = cv2.VideoCapture(0)

    # Diccionario 4x4 para Luis
    principal_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

    # Diccionario 5x5 para Luis
    luis_dict_5x5 = aruco.getPredefinedDictionary(aruco.DICT_5X5_50)

    # Diccionario 6x6 para Jaime
    jaime_dict_6x6 = aruco.getPredefinedDictionary(aruco.DICT_6X6_50)

    # Cargamos el clasificador de caras de OpenCV
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Cargar el video
    video_path = 'Img/SheidexPapure.mp4'  # Cambia esto por la ruta de tu video
    cap_overlay = cv2.VideoCapture(video_path)

    def get_next_overlay_frame():
        ret, frame = cap_overlay.read()
        if not ret:
            cap_overlay.set(cv2.CAP_PROP_POS_FRAMES, 0)
            ret, frame = cap_overlay.read()
        return frame

    def draw_augmented_overlay(pts_1, overlay_image, image):
        """Overlay the image 'overlay_image' onto the image 'image'"""

        # Define the squares of the overlay_image image to be drawn:
        pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                            [0, overlay_image.shape[0]]])

        # Draw border to see the limits of the image:
        cv2.rectangle(overlay_image, (0, 0), (overlay_image.shape[1], overlay_image.shape[0]), (255, 255, 0), 10)

        # Create the transformation matrix:
        M = cv2.getPerspectiveTransform(pts_2, pts_1)

        # Transform the overlay_image image using the transformation matrix M:
        dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))

        # Create the mask:
        dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)

        # Compute bitwise conjunction using the calculated mask:
        image_masked = cv2.bitwise_and(image, image, mask=mask)

        # Add the two images to create the resulting image:
        result = cv2.add(dst_image, image_masked)
        return result

    while True:
        ret, frame = cap.read()

        # Convertir la imagen a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar marcadores 4x4
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 15 in ids:
            bienvenida("Luis", frame)
            delay("Diccionario 5x5", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 5x5
                corners, ids, _ = aruco.detectMarkers(gray, luis_dict_5x5)

                if ids is not None and 10 in ids:
                    # Get the next frame from the overlay video
                    overlay_frame = get_next_overlay_frame()
                    if overlay_frame is not None:
                        # Overlay the video frame onto the camera frame
                        frame = draw_augmented_overlay(corners[0][0], overlay_frame, frame)

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 5x5", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        # Detectar marcadores 6x6 para Jaime
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 13 in ids:
            bienvenida("Jaime", frame)
            delay("Diccionario 6x6", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 6x6
                corners, ids, _ = aruco.detectMarkers(gray, jaime_dict_6x6)

                if ids is not None and 20 in ids:
                   # Aplicar blur en la cara
                    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)) #en vez de ser borrado, podria ser marcaras
                    for (x, y, w, h) in faces:
                        face = frame[y:y + h, x:x + w]
                        face = cv2.GaussianBlur(face, (99, 99), 30)
                        frame[y:y + h, x:x + w] = face


                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 6x6", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        cv2.imshow("Original", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
            break

    cap.release()
    cap_overlay.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


<h1>V6</h1>

<p>Ya muestra la imagen en el rostro, pero con un fondo negro, ya que se intento colocar un filtro blur y luego montar la imagen, pero no se pudo</p>

In [6]:
import cv2
import cv2.aruco as aruco
import numpy as np
import time

def bienvenida(name, frame):
    cv2.putText(img=frame, text=f"Bienvenido, {name} Acceso permitido.",
                org=(10, 30), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=0.6,
                color=(255, 255, 255), thickness=2)

def delay(window_name, frame, delay):
    cv2.imshow(window_name, frame)
    cv2.waitKey(delay * 1000)
    cv2.destroyAllWindows()

def overlay_video(pts_1, overlay_image, image):
    # Define the squares of the overlay_image image to be drawn:
    pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                        [0, overlay_image.shape[0]]])

    # Create the transformation matrix:
    M = cv2.getPerspectiveTransform(pts_2, pts_1)

    # Transform the overlay_image image using the transformation matrix M:
    dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))

    # Create the mask:
    dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)

    # Compute bitwise conjunction using the calculated mask:
    image_masked = cv2.bitwise_and(image, image, mask=mask)

    # Add the two images to create the resulting image:
    result = cv2.add(dst_image, image_masked)
    return result

def main():
    # Configuración de la cámara
    cap = cv2.VideoCapture(0)

    # Diccionario 4x4 para Luis
    principal_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

    # Diccionario 5x5 para Luis
    luis_dict_5x5 = aruco.getPredefinedDictionary(aruco.DICT_5X5_50)

    # Diccionario 6x6 para Jaime
    jaime_dict_6x6 = aruco.getPredefinedDictionary(aruco.DICT_6X6_50)

    # Cargamos el clasificador de caras de OpenCV
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Cargar el video
    video_path = 'Img/muchacho.mp4'  # Cambia esto por la ruta de tu video
    cap_overlay = cv2.VideoCapture(video_path)

    # Cargar la imagen para superponer
    overlay_image_path = 'Img/sigma.png'
    overlay_image = cv2.imread(overlay_image_path, cv2.IMREAD_UNCHANGED)
    overlay_image = cv2.resize(overlay_image, (100, 100))  # Ajusta el tamaño según sea necesario

    def get_next_overlay_frame():
        ret, frame = cap_overlay.read()
        if not ret:
            cap_overlay.set(cv2.CAP_PROP_POS_FRAMES, 0)
            ret, frame = cap_overlay.read()
        return frame

    def draw_augmented_overlay(pts_1, overlay_image, image):
        """Overlay the image 'overlay_image' onto the image 'image'"""

        # Define the squares of the overlay_image image to be drawn:
        pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                            [0, overlay_image.shape[0]]])

        # Draw border to see the limits of the image:
        cv2.rectangle(overlay_image, (0, 0), (overlay_image.shape[1], overlay_image.shape[0]), (255, 255, 0), 10)

        # Create the transformation matrix:
        M = cv2.getPerspectiveTransform(pts_2, pts_1)

        # Transform the overlay_image image using the transformation matrix M:
        dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))

        # Create the mask:
        dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)

        # Compute bitwise conjunction using the calculated mask:
        image_masked = cv2.bitwise_and(image, image, mask=mask)

        # Add the two images to create the resulting image:
        result = cv2.add(dst_image, image_masked)
        return result


        # Add the two images to create the resulting image:
        result = cv2.add(dst_image, image_masked)

        return result

    while True:
        ret, frame = cap.read()

        # Convertir la imagen a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar marcadores 4x4
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 15 in ids:
            bienvenida("Luis", frame)
            delay("Diccionario 5x5", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 5x5
                corners, ids, _ = aruco.detectMarkers(gray, luis_dict_5x5)

                if ids is not None and 10 in ids:
                    # Get the next frame from the overlay video
                    overlay_frame = get_next_overlay_frame()
                    if overlay_frame is not None:
                        # Overlay the video frame onto the camera frame
                        frame = draw_augmented_overlay(corners[0][0], overlay_frame, frame)

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 5x5", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        # Detectar marcadores 6x6 para Jaime
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 13 in ids:
            bienvenida("Jaime", frame)
            delay("Diccionario 6x6", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 6x6
                corners, ids, _ = aruco.detectMarkers(gray, jaime_dict_6x6)

                if ids is not None and 20 in ids:
                    # Aplicar blur en la cara
                    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                    for (x, y, w, h) in faces:
                        face = frame[y:y + h, x:x + w]
                        face = cv2.GaussianBlur(face, (99, 99), 30)
                        frame[y:y + h, x:x + w] = face

                        # Superponer la imagen en la cara
                        face_height, face_width, _ = face.shape
                        overlay_image_resized = cv2.resize(overlay_image, (face_width, face_height))
                        frame[y:y + h, x:x + w] = cv2.addWeighted(frame[y:y + h, x:x + w], 1.0, overlay_image_resized[:, :, :3], 0.7, 0)

                if ids is not None and 8 in ids:
                    # Aplicar blur en la cara
                    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                    for (x, y, w, h) in faces:
                        face = frame[y:y + h, x:x + w]
                        face = cv2.GaussianBlur(face, (99, 99), 30)
                        frame[y:y + h, x:x + w] = face

                        # Superponer la imagen en la cara
                        face_height, face_width, _ = face.shape
                        overlay_image_resized = cv2.resize(overlay_image, (face_width, face_height))
                        frame[y:y + h, x:x + w] = cv2.addWeighted(frame[y:y + h, x:x + w], 1.0, overlay_image_resized[:, :, :3], 0.7, 0)


                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 6x6", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        cv2.imshow("Original", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
            break

    cap.release()
    cap_overlay.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


In [17]:
import cv2
import cv2.aruco as aruco
import numpy as np
#import time

def bienvenida(name, frame):
    cv2.putText(img=frame, text=f"Bienvenido, {name} Acceso permitido.",
                org=(10, 30), fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=0.6,
                color=(255, 255, 255), thickness=2)

def delay(window_name, frame, delay):
    cv2.imshow(window_name, frame)
    cv2.waitKey(delay * 1000)
    cv2.destroyAllWindows()

def overlay_video(pts_1, overlay_image, image):
    # Define the squares of the overlay_image image to be drawn:
    pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                        [0, overlay_image.shape[0]]])

    # Create the transformation matrix:
    M = cv2.getPerspectiveTransform(pts_2, pts_1)

    # Transform the overlay_image image using the transformation matrix M:
    dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))

    # Create the mask:
    dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)

    # Compute bitwise conjunction using the calculated mask:
    image_masked = cv2.bitwise_and(image, image, mask=mask)

    # Add the two images to create the resulting image:
    result = cv2.add(dst_image, image_masked)
    return result

#FUNCION DE MASCARA
def meme(frame):
    # Load the cascade classifier for face detection:
    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")


    img_cuerno = cv2.imread('Img/meme.png', 1)  # Use 1 to read the image in color

    # Convert the frame to grayscale:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces using the 'detectMultiScale()' function:
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        
        x1 = x
        x2 = x + w
        y1 = y - h  
        y2 = y + h  

        
        cuerno_resized = cv2.resize(img_cuerno, (w, h))

        # Superpose the cuerno on the frame
        frame[y:y + h, x:x + w] = cv2.addWeighted(frame[y:y + h, x:x + w], 1, cuerno_resized, 1, 0)

    return frame

def main():
    # Configuración de la cámara
    cap = cv2.VideoCapture(0)

    # Diccionario 4x4 para Luis
    principal_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

    # Diccionario 5x5 para Luis
    luis_dict_5x5 = aruco.getPredefinedDictionary(aruco.DICT_5X5_50)

    # Diccionario 6x6 para Jaime
    jaime_dict_6x6 = aruco.getPredefinedDictionary(aruco.DICT_6X6_50)

    # Cargamos el clasificador de caras de OpenCV
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Cargar el video
    video_path = 'Img/muchacho.mp4'  # Cambia esto por la ruta de tu video
    cap_overlay = cv2.VideoCapture(video_path)

    # Cargar la imagen para superponer
    overlay_image_path = 'Img/sigma.png'
    overlay_image = cv2.imread(overlay_image_path, cv2.IMREAD_UNCHANGED)
    overlay_image = cv2.resize(overlay_image, (100, 100))  # Ajusta el tamaño según sea necesario


    def get_next_overlay_frame():
        ret, frame = cap_overlay.read()
        if not ret:
            cap_overlay.set(cv2.CAP_PROP_POS_FRAMES, 0)
            ret, frame = cap_overlay.read()
        return frame

    def draw_augmented_overlay(pts_1, overlay_image, image):
        """Overlay the image 'overlay_image' onto the image 'image'"""

        # Define the squares of the overlay_image image to be drawn:
        pts_2 = np.float32([[0, 0], [overlay_image.shape[1], 0], [overlay_image.shape[1], overlay_image.shape[0]],
                            [0, overlay_image.shape[0]]])

        # Draw border to see the limits of the image:
        cv2.rectangle(overlay_image, (0, 0), (overlay_image.shape[1], overlay_image.shape[0]), (255, 255, 0), 10)

        # Create the transformation matrix:
        M = cv2.getPerspectiveTransform(pts_2, pts_1)

        # Transform the overlay_image image using the transformation matrix M:
        dst_image = cv2.warpPerspective(overlay_image, M, (image.shape[1], image.shape[0]))

        # Create the mask:
        dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(dst_image_gray, 0, 255, cv2.THRESH_BINARY_INV)

        # Compute bitwise conjunction using the calculated mask:
        image_masked = cv2.bitwise_and(image, image, mask=mask)

        # Add the two images to create the resulting image:
        result = cv2.add(dst_image, image_masked)
        return result


        # Add the two images to create the resulting image:
        result = cv2.add(dst_image, image_masked)

        return result

    while True:
        ret, frame = cap.read()

        # Convertir la imagen a escala de grises
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar marcadores 4x4
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 15 in ids:
            bienvenida("Luis", frame)
            delay("Diccionario 5x5", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 5x5
                corners, ids, _ = aruco.detectMarkers(gray, luis_dict_5x5)

                if ids is not None and 10 in ids:
                    # Get the next frame from the overlay video
                    overlay_frame = get_next_overlay_frame()
                    if overlay_frame is not None:
                        # Overlay the video frame onto the camera frame
                        frame = draw_augmented_overlay(corners[0][0], overlay_frame, frame)

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 5x5", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        # Detectar marcadores 6x6 para Jaime
        corners, ids, _ = aruco.detectMarkers(gray, principal_dict)

        if ids is not None and 13 in ids:
            bienvenida("Jaime", frame)
            delay("Diccionario 6x6", frame, 5)

            while True:
                ret, frame = cap.read()
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar marcadores 6x6
                corners, ids, _ = aruco.detectMarkers(gray, jaime_dict_6x6)

                if ids is not None and 20 in ids:
                    # Aplicar blur en la cara
                    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
                    for (x, y, w, h) in faces:
                        face = frame[y:y + h, x:x + w]
                        face = cv2.GaussianBlur(face, (99, 99), 30)
                        frame[y:y + h, x:x + w] = face

                        # Superponer la imagen en la cara
                        face_height, face_width, _ = face.shape
                        overlay_image_resized = cv2.resize(overlay_image, (face_width, face_height))
                        frame[y:y + h, x:x + w] = cv2.addWeighted(frame[y:y + h, x:x + w], 1.0, overlay_image_resized[:, :, :3], 0.7, 0)

                elif ids is not None and 8 in ids:
                    meme(frame)

                elif ids is not None and 15 in ids:
                    cv2.destroyAllWindows()
                    break  # Volver a la ventana anterior

                cv2.imshow("Diccionario 6x6", frame)

                if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
                    break

        cv2.imshow("Original", frame)

        if cv2.waitKey(1) & 0xFF == 27:  # Presionar Esc para salir
            break

    cap.release()
    cap_overlay.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


